# Notebook 02: Element Extraction (Layer 2)

This notebook demonstrates extracting structured elements from PDFs:
- Text blocks (direct extraction + OCR fallback)
- Tables (pdfplumber for native, Docling/Surya for scanned)
- Images and engineering drawings (crop from rendered page)
- Hybrid page routing

In [ ]:
import sys
sys.path.insert(0, '..')

import fitz
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yaml
from pathlib import Path

from src.preprocessing.image_preprocessor import ImagePreprocessor
from src.classifier.page_classifier import PageClassifier
from src.classifier.region_classifier import RegionClassifier
from src.models.document_ir import DocumentElement, PageData, ParsedDocument
from src.models.classification_ir import RegionType

with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

preprocessor = ImagePreprocessor(config)
classifier = PageClassifier(config)
region_classifier = RegionClassifier(config)
print('All components loaded.')

In [ ]:
PDF_PATH = 'sample_pdfs/v1.pdf'
doc = fitz.open(PDF_PATH)
print(f'Document: {PDF_PATH}, Pages: {doc.page_count}')

## 2A. Direct Text Extraction (Machine-Readable Pages)

For pages with embedded fonts, extract text directly from the PDF structure.

In [ ]:
page = doc[0]

# Method 1: Plain text
plain_text = page.get_text('text')
print(f'Plain text ({len(plain_text)} chars):')
print(plain_text[:500])
print('...')

In [ ]:
# Method 2: Structured blocks with font info
blocks = page.get_text('dict')['blocks']

text_blocks = [b for b in blocks if b['type'] == 0]
image_blocks = [b for b in blocks if b['type'] == 1]

print(f'Text blocks: {len(text_blocks)}')
print(f'Image blocks: {len(image_blocks)}')

for i, block in enumerate(text_blocks[:5]):
    text = ''
    fonts = set()
    for line in block.get('lines', []):
        for span in line.get('spans', []):
            text += span.get('text', '')
            fonts.add(span.get('font', 'unknown'))
    bbox = tuple(int(x) for x in block['bbox'])
    print(f'  Block {i}: bbox={bbox}, fonts={fonts}')
    print(f'    Text: {text[:100]}...' if len(text) > 100 else f'    Text: {text}')

## 2B. Table Extraction with pdfplumber

In [ ]:
import pdfplumber

with pdfplumber.open(PDF_PATH) as pdf:
    for i, plumber_page in enumerate(pdf.pages):
        tables = plumber_page.extract_tables()
        if tables:
            print(f'\nPage {i}: Found {len(tables)} table(s)')
            for j, table in enumerate(tables):
                if table and len(table) > 1:
                    df = pd.DataFrame(table[1:], columns=table[0])
                    print(f'  Table {j}: {df.shape[0]} rows x {df.shape[1]} cols')
                    print(df.head())
                    print()

## 2C. Image Extraction

In [ ]:
page = doc[0]
images = page.get_images(full=True)
print(f'Page 0: {len(images)} embedded images')

for i, img_info in enumerate(images[:5]):
    xref = img_info[0]
    base_image = doc.extract_image(xref)
    if base_image:
        img_bytes = base_image['image']
        ext = base_image['ext']
        w = base_image['width']
        h = base_image['height']
        print(f'  Image {i}: {w}x{h}, format={ext}, size={len(img_bytes)} bytes')
        
        # Decode and display
        nparr = np.frombuffer(img_bytes, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        if img is not None:
            plt.figure(figsize=(6, 4))
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            plt.title(f'Embedded Image {i} ({w}x{h})')
            plt.axis('off')
            plt.show()

## 2D. Full Extraction Pipeline

Build a complete `ParsedDocument` from a PDF.

In [ ]:
def extract_document(pdf_path: str, config: dict) -> ParsedDocument:
    """Extract all elements from a PDF into a ParsedDocument."""
    doc = fitz.open(pdf_path)
    parsed = ParsedDocument(
        source_path=pdf_path,
        version_label=Path(pdf_path).stem,
        total_pages=doc.page_count,
    )
    
    full_text_parts = []
    
    for page in doc:
        # Classify
        cls = classifier.classify_page(page)
        
        # Extract text
        text = page.get_text('text').strip()
        full_text_parts.append(text)
        
        page_data = PageData(
            page_number=page.number,
            classification=cls,
            width=page.rect.width,
            height=page.rect.height,
        )
        
        # Add text element
        if text:
            page_data.elements.append(DocumentElement(
                element_type='text',
                page_number=page.number,
                bounding_box=(0, 0, page.rect.width, page.rect.height),
                content=text,
                region_type=RegionType.NATIVE_TEXT,
                extraction_strategy='direct',
                confidence=cls.confidence,
                reading_order=0,
            ))
        
        parsed.pages.append(page_data)
    
    parsed.raw_full_text = '\n\n'.join(full_text_parts)
    doc.close()
    return parsed

parsed = extract_document(PDF_PATH, config)
print(parsed.summary())

for page in parsed.pages:
    cls_type = page.classification.overall_type if page.classification else 'unknown'
    n_elements = len(page.elements)
    print(f'  Page {page.page_number}: {cls_type}, {n_elements} elements')

In [ ]:
print('Done! Element extraction pipeline validated.')